In [1]:
# !pip install --quiet  datasets #to access squad dataset
# !pip install --quiet pyarrow   #to deal with parquet files for saving dataset if required
# !pip install --quiet  tqdm     #for progress bars
# !pip install --quiet transformers # for t5 model
# !pip install --quiet tokenizers  #tokenizers from HuggingFace
# !pip install --quiet sentencepiece #subword tokenizer used by T5
# !pip install --quiet pytorch-lightning # pytorch wrapper 
# !pip install --quiet torchtext # text utilities

# Fetching Datasets

In [2]:
#imports
import pandas as pd
import torch
from tqdm import tqdm
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from pprint import pprint
import copy
import numpy as np
from collections import defaultdict
import ipdb
import random


pd.options.display.max_rows , pd.options.display.max_columns  = 100,100  

In [3]:
device  = 'cuda' if torch.cuda.is_available() else "cpu"
device

'cuda'

# Creating a Pytorch DataSet for T5 Training and Validation

In [4]:
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup,
    AutoTokenizer, 
    LongT5Model
) 

model = ["t5", "long-t5"]
size = ["base", "large", "xl"]
model_attention = ["local", "tglobal"]

model_idx = 0
size_idx = 0
model_idx = 0

model_max_length = None
# model_max_length = 16000

In [5]:
t5_tokenizer = AutoTokenizer.from_pretrained(f"google/{model[model_idx]}-{model_attention[model_idx]}-{size[size_idx]}", model_max_length=model_max_length)
t5_model = LongT5Model.from_pretrained(f"google/{model[model_idx]}-{model_attention[model_idx]}-{size[size_idx]}")

print(f"Max token lenght: {t5_tokenizer.model_max_length}")

OSError: google/t5-local-base is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.

In [6]:
class QuestionGenerationDataset(Dataset):
    def __init__(self, tokenizer, filepath, max_len_inp=512,max_len_out=96):
        self.path = filepath

        self.passage_column = "context"
        self.answer = "answer"
        self.question = "question"

        # self.data = pd.read_csv(self.path)
        self.data = pd.read_parquet(self.path).iloc[:2000,:]

        self.max_len_input = max_len_inp
        self.max_len_output = max_len_out
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []
        self._build()

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        src_mask = self.inputs[index]["attention_mask"].squeeze()  #squeeze to get rid of the batch dimension
        target_mask = self.targets[index]["attention_mask"].squeeze()  # convert [batch,dim] to [dim] 

        labels = copy.deepcopy(target_ids)
        labels [labels==0] = -100

        return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask,"labels":labels}

    def _build(self):
        for rownum,val in tqdm(self.data.iterrows()): # Iterating over the dataframe
            passage,answer,target = val[self.passage_column],val[self.answer],val[self.question]

#             input_ = f"context: {passage}  answer: {answer}" # T5 Input format for question answering tasks 
#             target = f"question: {str(target)}" # Output format we require
            
            input_ = f"context: {passage} question: {str(target)}" # T5 Input format for question answering tasks 
            target = f"answer: {answer}" # Output format we require
            
#             question_plus = f"answer_me: {str(question)}"
#             question_plus += f" context: {str(context)} </s>"

            # tokenize inputs
            tokenized_inputs = self.tokenizer.batch_encode_plus(
                [input_], max_length=self.max_len_input,padding='max_length',
                truncation = True,return_tensors="pt"
            )
            # tokenize targets
            tokenized_targets = self.tokenizer.batch_encode_plus(
                [target], max_length=self.max_len_output,padding='max_length',
                truncation = True,
                return_tensors="pt"
            )

            self.inputs.append(tokenized_inputs)
            self.targets.append(tokenized_targets)

In [7]:
# train_path = 'train_squad.parquet' # change this accordingly
# validation_path = 'validation_squad.parquet'
train_path = 'train_tdm.parquet' # change this accordingly
validation_path = 'dev_tdm.parquet'
train_dataset = QuestionGenerationDataset(t5_tokenizer, train_path)
validation_dataset = QuestionGenerationDataset(t5_tokenizer, validation_path)

147it [00:00, 353.93it/s]
63it [00:00, 354.12it/s]


In [8]:
# Data Sample

train_sample = train_dataset[50] # thanks to __getitem__
decoded_train_input = t5_tokenizer.decode(train_sample['source_ids'])
decoded_train_output = t5_tokenizer.decode(train_sample['target_ids'])

print(decoded_train_input)
print(decoded_train_output)

context: Adaptation of Deep Bidirectional Multilingual Transformers for Russian Language The paper introduces methods of adaptation of multilingual masked language models fora specific language. Pre-trained bidirectional language models show state-of-the-art performance on a wide range of tasks including reading comprehension, natural language inference, and sentiment analysis. At the moment there are two alternative approaches to train such models: monolingual and multilingual. While language specific models show superior performance, multilingual models allow to perform a transfer from one language to another and solve tasks for different languages simultaneously. This work shows that transfer learning from a multilingual model to monolingual model results in significant growth of performance on such tasks as reading comprehension, paraphrase detection, and sentiment analysis. Furthermore, multilingual initialization of monolingual model substantially reduces training time. Pre-train

# Fine Tuning T5

In [9]:
import pytorch_lightning as pl
from torch.optim import AdamW
import argparse
from transformers import (
    get_linear_schedule_with_warmup
  )

class T5Tuner(pl.LightningModule):

    def __init__(self,t5model, t5tokenizer,batchsize=4):
        super().__init__()
        self.model = t5model
        self.tokenizer = t5tokenizer
        self.batch_size = batchsize

    def forward( self, input_ids, attention_mask=None, 
                decoder_attention_mask=None, 
                lm_labels=None):
      
         outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_attention_mask=decoder_attention_mask,
            labels=lm_labels,
        )
         
         return outputs

    def training_step(self, batch, batch_idx):
        outputs = self.forward(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_attention_mask=batch['target_mask'],
            lm_labels=batch['labels']
        )

        loss = outputs[0]
        self.log('train_loss',loss)
        return loss

    def validation_step(self, batch, batch_idx):
        outputs = self.forward(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_attention_mask=batch['target_mask'],
            lm_labels=batch['labels']
        )

        loss = outputs[0]
        self.log("val_loss",loss)
        return loss

    def train_dataloader(self):
        return DataLoader(train_dataset, batch_size=self.batch_size,
                          num_workers=2)

    def val_dataloader(self):
        return DataLoader(validation_dataset, 
                          batch_size=self.batch_size,
                          num_workers=2)

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=3e-4, eps=1e-8)
        return optimizer

In [10]:
device

'cuda'

In [11]:
model = T5Tuner(t5_model, t5_tokenizer)

trainer = pl.Trainer(max_epochs = 3,
#                      accelerator=device, 
                     gpus=1)

trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 60.5 M
-----------------------------------------------------
60.5 M    Trainable params
0         Non-trainable params
60.5 M    Total params
242.028   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/nfs/home/kabenamualus/.local/lib/python3.6/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 48 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/nfs/home/kabenamualus/.local/lib/python3.6/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 48 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/nfs/home/kabenamualus/.local/lib/python3.6/site-packages/pytorch_lightning/trainer/data_loading

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [13]:
# # saving the model
# !mkdir "t5_tokenizer"
# !mkdir "t5_trained_model"
model.model.save_pretrained('t5_trained_tdm_model_QA')
t5_tokenizer.save_pretrained('t5_tokenizertdm_QA')

('t5_tokenizertdm_QA/spiece.model',
 't5_tokenizertdm_QA/special_tokens_map.json',
 't5_tokenizertdm_QA/added_tokens.json')

# Inference / Predictions

In [14]:
trained_model_path = 't5_trained_tdm_model_QA'
trained_tokenizer = 't5_tokenizertdm_QA'
device = 'cpu'

In [15]:
model = T5ForConditionalGeneration.from_pretrained(trained_model_path)
tokenizer = T5Tokenizer.from_pretrained(trained_tokenizer)

Text Sample

In [16]:
# context ="President Donald Trump said and predicted that some states would reopen this month."
# # answer = "Donald Trump"
# question = "Who is the pre"
# text = "context: "+context + " " + "answer: " + answer
# print(text)

In [40]:
idx = random.randint(0, 50)
idx

38

In [66]:
# Data Sample

idx = random.randint(0, 130)
idx = 130

# train_sample = validation_dataset[idx] # thanks to __getitem__
train_sample = train_dataset[idx] # thanks to __getitem__
decoded_train_input = t5_tokenizer.decode(train_sample['source_ids'])
decoded_train_output = t5_tokenizer.decode(train_sample['target_ids'])

print(decoded_train_input)
print(decoded_train_output)

context: GRADIENT PENALTY FROM A MAXIMUM MARGIN PER- SPECTIVE A popular heuristic for improved performance in Generative adversarial networks (GANs) is to use some form of gradient penalty on the discriminator. This gradient penalty was originally motivated by a Wasserstein distance formulation. However, the use of gradient penalty in other GAN formulations is not well motivated. We present a unifying framework of expected margin maximization and show that a wide range of gradient-penalized GANs (e.g., Wasserstein, Standard, Least-Squares, and Hinge GANs) can be derived from this framework. Our results imply that employing gradient penalties induces a large-margin classifier (thus, a large-margin discriminator in GANs). We describe how expected margin maximization helps reduce vanishing gradients at fake (generated) samples, a known problem in GANs. From this framework, we derive anew L  ⁇  gradient norm penalty with Hinge loss which generally produces equally good (or better) generate

In [70]:
# context = "Adaptation of Deep Bidirectional Multilingual Transformers for Russian Language The paper introduces methods of adaptation of multilingual masked language models fora specific language. Pre-trained bidirectional language models show state-of-the-art performance on a wide range of tasks including reading comprehension, natural language inference, and sentiment analysis. At the moment there are two alternative approaches to train such models: monolingual and multilingual. While language specific models show superior performance, multilingual models allow to perform a transfer from one language to another and solve tasks for different languages simultaneously. This work shows that transfer learning from a multilingual model to monolingual model results in significant growth of performance on such tasks as reading comprehension, paraphrase detection, and sentiment analysis. Furthermore, multilingual initialization of monolingual model substantially reduces training time. Pre-trained models for the Russian language are open sourced. Table 1: ParaPhraser. We compare BERT based models with models in non- standard run setting, when all resources were allowed. Accuracy F - 1 Table 2: RuSentiment. We used only randomly selected posts (21,268) subset for training. Precision Recall F - 1 - Table 3: Results on question answering with SDSJ Task B. Models performance was evaluated on development set (public leaderboard subset). F - 1 ( dev ) EM ( dev ) question: Which Task | Datasets | Metric are in this article"
context = decoded_train_input
question = "Which Task # Datasets # Metric are in this article"


text = "context: "+context + " " + "question: " + question
# text = "context: "+context + " " + "answer: " + answer

encoding = tokenizer.encode_plus(text,max_length =512,padding='max_length', 
                                 truncation = True,
                                 return_tensors="pt").to(device)
print (encoding.keys())
input_ids,attention_mask  = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

model.eval()
beam_outputs = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_length=72, # How long the generated questions should be
    early_stopping=True,
    num_beams=5,
    num_return_sequences=2
)

for beam_output in beam_outputs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    print(sent)


dict_keys(['input_ids', 'attention_mask'])
answer: | Image Classification; CIFAR-10; FID |
answer: | Image Generation; CIFAR-10; FID |


dict_keys(['input_ids', 'attention_mask'])


answer: | Image Classification; CIFAR-10; FID |
answer: | Image Generation; CIFAR-10; FID |


# Deployment Demo

In [ ]:
!pip install --quiet gradio==3.9

In [ ]:
def get_question(sentence,answer,mdl,tknizer):

  ''' function to generate questions. Takes a sentence,answer,
      model and tokenizer 
  '''

  text = "context: {} answer: {}".format(sentence,answer)
  print (text)
  max_len = 256
  encoding = tknizer.encode_plus(text,max_length=max_len, pad_to_max_length=False,truncation=True, return_tensors="pt")

  input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

  outs = mdl.generate(input_ids=input_ids,
                                  attention_mask=attention_mask,
                                  early_stopping=True,
                                  num_beams=5,
                                  num_return_sequences=1,
                                  no_repeat_ngram_size=2,
                                  max_length=72)


  dec = [tknizer.decode(ids,skip_special_tokens=True) for ids in outs]


  Question = dec[0].replace("question:","")
  Question= Question.strip()
  return Question

In [ ]:
context = "Donald Trump is an American media personality and businessman who served as the 45th president of the United States."
answer = "Donald Trump"

ques = get_question(context,answer,model,tokenizer)
print ("question: ",ques)

context: Donald Trump is an American media personality and businessman who served as the 45th president of the United States. answer: Donald Trump
question:  Who served as the 45th president of the United States?


In [ ]:
import gradio as gr

context = gr.inputs.Textbox(lines=5,placeholder="Enter paragraph/context here...")
answer = gr.inputs.Textbox(lines=3, placeholder="Enter answer/keyword here...")
question = gr.outputs.Textbox( type="auto", label="Question")

def generate_question(context,answer):
  return get_question(context,answer,model,tokenizer)

iface = gr.Interface(
  fn=generate_question, 
  inputs=[context,answer], 
  outputs=question)

iface.launch(debug=False,share=True)

/usr/local/lib/python3.8/dist-packages/gradio/inputs.py:26: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.8/dist-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.8/dist-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)
/usr/local/lib/python3.8/dist-packages/gradio/outputs.py:21: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


IMPORTANT: You are using gradio version 3.9, however version 3.14.0 is available, please upgrade.
--------
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://7c67647ce4455126.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x7fc62ea10df0>,
 'http://127.0.0.1:7860/',
 'https://7c67647ce4455126.gradio.app')